In [ ]:
def extract_data():

    drive.mount('/content/G_drive')

    f = open('file_name', 'r')
    raw = f.read(1000000)
    f.close()
    return raw


In [ ]:
def preprocessing(raw):
    raw = raw.replace('\n', ' ')
    raw = raw.replace('\xa0', ' ')
    raw = raw.replace("\\'", "'")

    # perform sentence tokenization and add START and END tokens
    sentences = sent_tokenize(raw)
    sentences = ['START_TAG ' + s + ' END_TAG' for s in sentences]

    random.shuffle(sentences)
    # split the data into train+dev and test
    train_dev = sentences[: int(len(sentences) * 0.9)]
    test = sentences[int(len(sentences) * 0.9):]
    return train_dev, test


In [ ]:
# function to split the train+dev data into train and dev
def split_train_dev(train_dev):
    # shuffle the data
    random.shuffle(train_dev)
    train = train_dev[: int(len(train_dev) * 0.9)]
    dev = train_dev[int(len(train_dev) * 0.9):]
    # delete the original train+dev to save memory
    del train_dev
    return train, dev


In [ ]:
# function to make ngram dictionary
# sentences is a list of sentences
# n is the n in ngram
def make_ngram_dict(sentences, n):
    ngram_dict = {}
    for s in sentences:
        words = s.split()
        ngram = ngrams(words, n)
        # convert ngram into string
        ngram = [' '.join(grams) for grams in ngram]

        for g in ngram:
            # if n == 3:
              # print(g)
            if g in ngram_dict:
                ngram_dict[g] += 1
            else:
                ngram_dict[g] = 1
    return ngram_dict


In [ ]:
# return the threshold value for the ngram dictionary and percentage
# items in the dictionary that are below the threshold value should be less than percentage of the total number of items in the dictionary
def calculate_threshold(ngram_dict, percentage):
    total = sum(ngram_dict.values())
    ngram_dict = sorted(ngram_dict.items(), key=lambda x: x[1])
    threshold = 1
    count = 0
    for i in range(len(ngram_dict)):
        count += ngram_dict[i][1]
        if (100 * count) / total > percentage:
            threshold = ngram_dict[i][1]
            break
    return threshold + 1


In [ ]:
# function to convert the less frequent tokens into UNK_TAG
# sentences is a list of sentences
# threshold is the threshold of the frequency of a token
def convert_unk(sentences,  threshold):
    # find the tokens that are less frequent than the threshold
    unigram_dict = make_ngram_dict(sentences, 1)
    less_freq_tokens = []
    for k, v in unigram_dict.items():
        if v < threshold:
            less_freq_tokens.append(k)

    print('SOME LESS FREQUENCY TOKENS, TO BE LABELLED UNKNOWN: ', less_freq_tokens[:10])

    # replace the less frequent tokens with UNK_TAG
    for i in range(len(sentences)):
        # do not use split(), instead use nltk.word_tokenize()
        words = nltk.word_tokenize(sentences[i])
        for j in range(len(words)):
            if words[j] in less_freq_tokens:
                words[j] = 'UNK_TAG'
        sentences[i] = ' ' .join(words)
    return sentences


In [ ]:
# convert the OOV tokens into UNK_TAG
def convert_oov(sentences, unigram_dict):
    # find the OOV tokens
    oov_tokens = []
    for s in sentences:
        words = nltk.word_tokenize(s)
        for w in words:
            #if the frequency of the token is 0, it is OOV
            if unigram_dict.get(w, 0) == 0:
                oov_tokens.append(w)

    # replace the OOV tokens with UNK_TAG
    for i in range(len(sentences)):
        words = nltk.word_tokenize(sentences[i])
        for j in range(len(words)):
            if words[j] in oov_tokens:
                words[j] = 'UNK_TAG'
        sentences[i] = ' ' .join(words)
    return sentences


In [ ]:
def calc_perplexity_log_likelihood(prob, M):

    perlexity = 0
    log_likelihood = 0

    for gram in prob:
        log_likelihood += np.log2(prob[gram] if prob[gram] > 0 else 1)

    perlexity = 2 ** (-log_likelihood / M)
    return perlexity, log_likelihood


def calc_perplexity_log_likelihood_degugger(prob, M):

    perlexity = 0
    log_likelihood = 0
    counter = 0
    for gram in prob:
        counter += 1
        if counter < 30:
          print('DEBUG: ', np.log2(prob[gram] if prob[gram] > 0 else 1))
        log_likelihood += np.log2(prob[gram] if prob[gram] > 0 else 1)

    perlexity = 2 ** (-log_likelihood / M)
    return perlexity, log_likelihood



In [ ]:
# linear interpolation


# function to calculate the probability with given lambdas and the model using linear interpolation. Return the dictionary of trigram probability
# lambda1, lambda2, lambda3 are the lambdas
# train is a list of sentences
def calc_prob_interpolation(lambda1, lambda2, lambda3, dev, unigram_dict, bigram_dict, trigram_dict):

    prob = {}
    # create a trigram dev list using different sentences of dev
    trigram_dev = []
    trigram_dict_dev = make_ngram_dict(dev, 3)

    for gram in trigram_dict_dev:
        trigram_dev.append(gram)
    for trigram_string in trigram_dev:
        words_in_trigram = trigram_string.split()
        
        bigram_string = ' ' .join(words_in_trigram[1:])
        
        unigram_string = words_in_trigram[2]
        p1 = trigram_dict.get(trigram_string, 0) / \
            bigram_dict.get(bigram_string, 1)
        p2 = bigram_dict.get(bigram_string, 0) / \
            unigram_dict.get(unigram_string, 1)
        p3 = unigram_dict.get(unigram_string, 0) / len(unigram_dict)
        prob[trigram_string] = lambda1 * p1 + lambda2 * p2 + lambda3 * p3

    
    return prob


def interpolation_smoothing(unigram_dict_train, bigram_dict_train, trigram_dict_train, dev):
    best_lambda1 = 0
    best_lambda2 = 0
    best_lambda3 = 0
    best_log_likelihood = -np.inf
    best_perplexity = np.inf
    best_prob = {}

    # calculate number of trigrams in the dev set
    M = 0
    for s in dev:
        M += len(word_tokenize(s))

    for lambda1 in np.arange(0, 1, 0.025):
        for lambda2 in np.arange(0, 1 - lambda1, 0.025):

            lambda3 = 1 - lambda1 - lambda2
            if lambda3 == 0:
              continue

            prob = calc_prob_interpolation(
                lambda1, lambda2, lambda3, dev, unigram_dict_train, bigram_dict_train, trigram_dict_train)
            perplexity, log_likelihood = calc_perplexity_log_likelihood(
                prob, M)
            #print(perplexity, log_likelihood)
            if log_likelihood > best_log_likelihood:
                best_lambda1 = lambda1
                best_lambda2 = lambda2
                best_lambda3 = lambda3
                best_log_likelihood = log_likelihood
                best_perplexity = perplexity
                best_prob = prob

    return best_lambda1, best_lambda2, best_lambda3, best_perplexity, best_log_likelihood 


In [ ]:
def calc_prob_discounting_bigram(bigram_dict_train, unigram_dict_train, beta1):
  prob = {}

  for unigram in unigram_dict_train:
    missing_mass = 0
    deno = 0
    bigram_count_zero = []
    for unigram2 in unigram_dict_train:
      bigram = unigram+ " " + unigram2
      if bigram in bigram_dict_train.keys():
        prob[bigram] = bigram_dict_train.get(bigram, 0) / unigram_dict_train.get(unigram , 1)
        missing_mass += beta1
      else:
        deno += unigram_dict_train.get(unigram2, 0)
        bigram_count_zero.append(unigram2)

    missing_mass /= unigram_dict_train.get(unigram, 1)
    for unigram2 in bigram_count_zero:
      bigram = unigram+ " " + unigram2
      prob[bigram] = missing_mass * unigram_dict_train.get(unigram2, 0) / deno

  return prob

In [ ]:
#Discounting Smoothing
def calc_prob_discounting(trigram_dict_train, bigram_dict_train, unigram_dict_train, dev, beta1, beta2):
    prob = {}
    bigram_prob  = calc_prob_discounting_bigram(bigram_dict_train, unigram_dict_train, beta1)
    missing_mass = 0
    trigram_dev = []
    trigram_dict_dev = make_ngram_dict(dev, 3)
    for gram in trigram_dict_dev:
        trigram_dev.append(gram)

    temp_prob = {}
    for bigram in bigram_dict_train:
      if bigram.split()[1] == 'END_TAG':
        continue
      missing_mass = 0
      deno = 0
      trigram_count_zero = []
      for unigram in unigram_dict_train:
        trigram = bigram+ " " + unigram
        if trigram in trigram_dict_train.keys():
          temp_prob[trigram] = trigram_dict_train.get(trigram, 0) / bigram_dict_train.get(bigram , 1)
          missing_mass += beta2
        else:
          bigram2=  " ".join(trigram.split()[1:])
          deno += bigram_prob.get(bigram2, 0)
          #print('DEBUG: DENO = ',deno, 'BIGRAM2 = ', bigram2 in bigram_prob.keys(),'BIGRAM2 =', bigram2)
          trigram_count_zero.append(unigram)

      missing_mass /= bigram_dict_train.get(bigram, 1)
      for unigram in trigram_count_zero:
        trigram = bigram + " " + unigram
        bigram2 = " ".join(trigram.split()[1:])

        temp_prob[trigram] = missing_mass * bigram_prob.get(bigram2, 0) / deno

    for trigram_string in trigram_dev:
      prob[trigram_string] = temp_prob.get(trigram_string, unigram_dict_train.get(trigram_string.split()[2] ,1) / sum(unigram_dict_train.values()))

    return prob   



In [ ]:
# laplace smoothing
# returns the dictionary of trigram probability

def calc_prob_laplace(trigram_dict_train, bigram_dict_train, unigram_dict_train, dev):
    # create a trigram dev list using different sentences of dev
    trigram_dev = []
    trigram_dict_dev = make_ngram_dict(dev, 3)
    for gram in trigram_dict_dev:
        trigram_dev.append(gram)

    prob = {}
    # for each trigram in the dev set, calculate the probability using add-one smoothing
    for trigram_string in trigram_dev:
        words_in_trigram = trigram_string.split()
        bigram_string = ' '.join(words_in_trigram[:1])
        count1 = trigram_dict_train.get(trigram_string, 0)
        count2 = bigram_dict_train.get(bigram_string, 0)
        prob[trigram_string] = (count1 + 1) / \
            (count2 + len(unigram_dict_train))

    return prob


def laplace_smoothing(unigram_dict_train, bigram_dict_train, trigram_dict_train, dev):
    prob = calc_prob_laplace(
        trigram_dict_train, bigram_dict_train, unigram_dict_train, dev)
    M = 0
    for s in dev:
        M += len(word_tokenize(s)) 
    perplexity, log_likelihood = calc_perplexity_log_likelihood(prob, M)
    return perplexity, log_likelihood


In [ ]:
# byte pair encoding
def byte_pair_encoding(train):


    vocab = {}
    temp = ""
    
    words = []
    # create word list from train
    for s in train:
        words.extend(word_tokenize(s))


    word_freq_dict = collections.defaultdict(int)

    for word in words:
        word = (' ').join(list(word))
        word_freq_dict[word] += 1

    #print(word_freq_dict)

    # word_freq_dict

    char_freq_dict = collections.defaultdict(int)
    for word, freq in word_freq_dict.items():
        chars = word.split()
        for char in chars:
            char_freq_dict[char] += freq




    # find the best pair

    def get_pairs(word_freq_dict):
        pairs = collections.defaultdict(int)
        #print('1', word_freq_dict)
        for word, freq in word_freq_dict.items():
            chars = word.split()
            for i in range(len(chars)-1):

                pairs[chars[i], chars[i+1]] += freq
                #print(chars[i], chars[i+1])
        return pairs


    def merge_byte_pairs(best_pair, word_freq_dict):
        # print(best_pair)
        merged_dict = {}
        bigram = re.escape(' '.join(best_pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in word_freq_dict:
            # print(word)
            w_out = p.sub(''.join(best_pair), word)
            merged_dict[w_out] = word_freq_dict[word]
        #print(merged_dict)
        return merged_dict


    def get_subword_tokens(word_freq_dict):
        char_freq_dict = collections.defaultdict(int)
        for word, freq in word_freq_dict.items():
            chars = word.split()
            for char in chars:
                char_freq_dict[char] += freq
        return char_freq_dict


    for i in range(300):
        pairs = get_pairs(word_freq_dict)
        #print(pairs)
        best_pair = max(pairs, key= lambda x: pairs[x])
        #print(best_pair)
        word_freq_dict = merge_byte_pairs(best_pair, word_freq_dict)
        subword_tokens = get_subword_tokens(word_freq_dict)

    return subword_tokens

In [ ]:
def tokenise(word, subword_tokens):
    if word in subword_tokens:
        return [word]
    start = 0
    tokens = []
    while start < len(word):
        for end in range(len(word), start, -1):
            subword = word[start:end]
            if subword in subword_tokens:
                tokens.append(subword)
                start = end
                break
        else:
            start += 1
    return tokens

In [ ]:
#returns the modified list of sentences train, that contains subwords instead of words
def tokenize_train(train, subword_tokens):
    tokenized_train_sentences = []
    for sentence in train:
        tokenized_sentence = []
        for word in word_tokenize(sentence):
            tokenized_sentence.extend(tokenise(word, subword_tokens))
        tokenized_train_sentences.append(' '.join(tokenized_sentence))
    return tokenized_train_sentences

In [ ]:
def execute():

    raw = extract_data()
    train_dev, test = preprocessing(raw)
    # print the number of sentences in the training and development set
    print('Number of sentences in the training and development set: ', len(train_dev))
    # print the number of sentences in the test set
    print('Number of sentences in the test set: ', len(test))

    print('')
    print('Part 1: Word-level language modeling')
    print('')

    for i in range(5):

        print('*'*50)
        print('Round ', i + 1)

        # initialize the data for this iteration
        train, dev = split_train_dev(train_dev)
        # print the number of sentences in the training set
        print('Number of sentences in the training set: ', len(train))
        # print the number of sentences in the development set
        print('Number of sentences in the development set: ', len(dev))

        unigram_dict_train = make_ngram_dict(train, 1)

        bigram_dict_train = make_ngram_dict(train, 2)
        trigram_dict_train = make_ngram_dict(train, 3)
        # for item in trigram_dict_train:
        # print(item[0])

        threshold = 2
        threshold = calculate_threshold(unigram_dict_train, 10)
        print('Threshold value for frequency: ', threshold)
        train = convert_unk(train, threshold)
        dev = convert_oov(dev, unigram_dict_train)
        test_new = convert_oov(test, unigram_dict_train)
        m_test = 0
        for s in test_new:
            m_test += len(word_tokenize(s))

        m_dev = 0
        for s in dev:
            m_dev += len(word_tokenize(s))

        print('-'*50)

        print('Linear interpolation')
        print(' ')
        lambda1, lambda2, lambda3, perplexity_interpolation, log_likelihood_interpolation = interpolation_smoothing(
            unigram_dict_train, bigram_dict_train, trigram_dict_train, dev)

        print('lambda1: ', lambda1)
        print('lambda2: ', lambda2)
        print('lambda3: ', lambda3)
        print('Perplexity: ', perplexity_interpolation)
        print('Log likelihood: ', log_likelihood_interpolation)
        print(' ')

        print('-'*50)

        # print('Backoff smoothing')
        # print(' ')
        # dev_prob_disco = calc_prob_discounting(trigram_dict_train, bigram_dict_train, unigram_dict_train, dev, 0.25, 0.25)
        # perplexity_disco_dev, log_likelihood_disco_dev = calc_perplexity_log_likelihood(dev_prob_disco, m_dev)
        # del dev_prob_disco
        # print('Perplexity: ', perplexity_disco_dev)
        # print('Log likelihood: ', log_likelihood_disco_dev)
        # print(' ')

        # print('-'*50)

        print('Laplace smoothing')
        print(' ')
        perplexity_laplace, log_likelihood_laplace = laplace_smoothing(
            unigram_dict_train, bigram_dict_train, trigram_dict_train, dev)
        print('Perplexity: ', perplexity_laplace)
        print('Log likelihood: ', log_likelihood_laplace)
        print(' ')

        print('-'*50)

        print('Test Set Performance Comparsion')
        test_prob_interpolation = calc_prob_interpolation(lambda1, lambda2, lambda3, test_new, unigram_dict_train, bigram_dict_train, trigram_dict_train)
        #m is the number of words in the new test set

        
        perplexity_interpolation_test, log_likelihood_interpolation_test = calc_perplexity_log_likelihood(test_prob_interpolation, m_test)
        del test_prob_interpolation

        test_prob_laplace = calc_prob_laplace(trigram_dict_train, bigram_dict_train, unigram_dict_train, test_new)
        perplexity_laplace_test, log_likelihood_laplace_test = calc_perplexity_log_likelihood(test_prob_laplace, m_test)
        del test_prob_laplace

        # test_prob_disco = calc_prob_discounting(trigram_dict_train, bigram_dict_train, unigram_dict_train, test_new, 0.25, 0.25)
        # perplexity_disco_test, log_likelihood_disco_test = calc_perplexity_log_likelihood(test_prob_disco, m_test)
        # del test_prob_disco
        
        print(' ')
        print('Interpolation: Perplexity = ' ,perplexity_interpolation_test, '  Log Likelihood = ', log_likelihood_interpolation_test)
        print('Laplace: Perplexity = ' ,perplexity_laplace_test, '  Log Likelihood = ', log_likelihood_laplace_test)
        # print('Backoff: Perplexity = ' ,perplexity_disco_test, '  Log Likelihood = ', log_likelihood_disco_test)

        del unigram_dict_train
        del bigram_dict_train
        del trigram_dict_train
        del train
        del dev
        del threshold

        print(' '*50)
        print(' '*50)

    print(':'*50)
    print('')

    print('')
    print('Part 2: Subword-level language modeling')
    print('')

    for i in range(5):

        print('*'*50)
        print('Round ', i + 1)

        # initialize the data for this iteration
        train, dev = split_train_dev(train_dev)
        # print the number of sentences in the training set
        print('Number of sentences in the training set: ', len(train))
        # print the number of sentences in the development set
        print('Number of sentences in the development set: ', len(dev))



        subwords = byte_pair_encoding(train)
        #print(subwords)
        train = tokenize_train(train, subwords)
        dev = tokenize_train(dev, subwords)
        tokenized_test = tokenize_train(test, subwords)

        m_test = 0
        for s in tokenized_test:
            m_test += len(word_tokenize(s))

        m_dev = 0
        for s in dev:
            m_dev += len(word_tokenize(s))

        unigram_dict_train = make_ngram_dict(train, 1)

        bigram_dict_train = make_ngram_dict(train, 2)
        trigram_dict_train = make_ngram_dict(train, 3)

        #print('Sample train set: ', train[:20])
        
        print('-'*50)

        print('Linear interpolation')
        print(' ')
        lambda1, lambda2, lambda3, perplexity_interpolation, log_likelihood_interpolation = interpolation_smoothing(
            unigram_dict_train, bigram_dict_train, trigram_dict_train, dev)

        print('lambda1: ', lambda1)
        print('lambda2: ', lambda2)
        print('lambda3: ', lambda3)
        print('Perplexity: ', perplexity_interpolation)
        print('Log likelihood: ', log_likelihood_interpolation)
        print(' ')


        print('-'*50)

        # print('Backoff smoothing')
        # print(' ')
        # dev_prob_disco = calc_prob_discounting(trigram_dict_train, bigram_dict_train, unigram_dict_train, dev, 0.25, 0.25)
        # perplexity_disco_dev, log_likelihood_disco_dev = calc_perplexity_log_likelihood(dev_prob_disco, m_dev)
        # del dev_prob_disco
        # print('Perplexity: ', perplexity_disco_dev)
        # print('Log likelihood: ', log_likelihood_disco_dev)
        # print(' ')

        # print('-'*50)


        print('Laplace smoothing')
        print(' ')
        perplexity_laplace, log_likelihood_laplace = laplace_smoothing(
            unigram_dict_train, bigram_dict_train, trigram_dict_train, dev)
        print('Perplexity: ', perplexity_laplace)
        print('Log likelihood: ', log_likelihood_laplace)
        print(' ')

        print('-'*50)

        print('Test Set Performance Comparsion')
        test_prob_interpolation = calc_prob_interpolation(lambda1, lambda2, lambda3, tokenized_test, unigram_dict_train, bigram_dict_train, trigram_dict_train)
        
        perplexity_interpolation_test, log_likelihood_interpolation_test = calc_perplexity_log_likelihood(test_prob_interpolation, m_test)
        del test_prob_interpolation

        test_prob_laplace = calc_prob_laplace(trigram_dict_train, bigram_dict_train, unigram_dict_train, tokenized_test)
        perplexity_laplace_test, log_likelihood_laplace_test = calc_perplexity_log_likelihood(test_prob_laplace, m_test)
        del test_prob_laplace

        # test_prob_disco = calc_prob_discounting(trigram_dict_train, bigram_dict_train, unigram_dict_train, tokenized_test, 0.25, 0.25)
        # perplexity_disco_test, log_likelihood_disco_test = calc_perplexity_log_likelihood(test_prob_disco, m_test)
        # del test_prob_disco

        print(' ')
        print('Interpolation: Perplexity = ' ,perplexity_interpolation_test, '  Log Likelihood = ', log_likelihood_interpolation_test)
        print('Laplace: Perplexity = ' ,perplexity_laplace_test, '  Log Likelihood = ', log_likelihood_laplace_test)
        # print('Backoff: Perplexity = ' ,perplexity_disco_test, '  Log Likelihood = ', log_likelihood_disco_test)
        
        
        del unigram_dict_train
        del bigram_dict_train
        del trigram_dict_train
        del train
        del dev

        print(' '*50)
        print(' '*50)




In [ ]:
import numpy as np
import random
import collections
import re
import nltk
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from google.colab import drive
nltk.download('punkt')
execute()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
Number of sentences in the training and development set:  6351
Number of sentences in the test set:  706

Part 1: Word-level language modeling

**************************************************
Round  1
Number of sentences in the training set:  5715
Number of sentences in the development set:  636
Threshold value for frequency:  2
SOME LESS FREQUENCY TOKENS, TO BE LABELLED UNKNOWN:  ['Ebeling,', 'late-18th-century', 'remarks', '(22', '1819', '1880;', 'alternatively', '"Mary', 'Anne"', '"Marian"),']
--------------------------------------------------
Linear interpolation
 
lambda1:  0.025
lambda2:  0.17500000000000002
lambda3:  0.7999999999999999
Perplexity:  51.280745543724024
Log likelihood:  -104132.09061850791
 
--------------------------------------------------
Laplace smoothing
 
Perplexity:  4213.339111784952
Log likelihood:  -220730.9982196913
 
--------------------------------------------------
Test Set Performance Comparsion
 
Interpolation: Perplexit